# ENGR-E 533: Deep Learning Systems
## Homework 2

### Khandokar Md. Nayem (knayem@iu.edu)
### Apr 3, 2018

### Import necessary files and set environment parameters
My assigned Node is `r-005` and GPU `0`.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.utils.np_utils import to_categorical

import tensorflow as tf
import numpy as np
import librosa
import glob

Using TensorFlow backend.


### Global Variables

Directory names, file formates.

In [2]:
PATH_directory = '/N/u/knayem/data/timit-homework/'

PATH_train = 'tr/'
PATH_val = 'v/'
PATH_test = 'te/'
PATH_denoise = 'te/'

CLEAN_format_train = 'trs*.wav'
NOISE_format_train = 'trn*.wav'
MIX_format_train = 'trx*.wav'

CLEAN_format_val = 'vs*.wav'
NOISE_format_val = 'vn*.wav'
MIX_format_val = 'vx*.wav'

MIX_format_test = 'tex*.wav'

Max_RNN = 5 # Number of Time Stamps in a RNN 

### File Names

Since there are huge number of files for both training, validation and testing, for convinience we write them in files. These are the name of the file.

In [3]:
TRAIN_S = 'TRAIN_S.txt'
TRAIN_N = 'TRAIN_N.txt'
TRAIN_X_cmplx = 'TRAIN_X_cmplx.txt'
TRAIN_X = 'TRAIN_X.txt'

VAL_S = 'VAL_S.txt'
VAL_S_cmplx = 'VAL_S_cmplx.txt'
VAL_N = 'VAL_N.txt'
VAL_X_cmplx = 'VAL_X_cmplx.txt'
VAL_X = 'VAL_X.txt'


TEST_S = 'TEST_S.txt'
TEST_N = 'TEST_N.txt'
TEST_X_cmplx = 'TEST_X_cmplx.txt'
TEST_X = 'TEST_X.txt'


### Load a .wav file
A function to get the Magnitude Spectum of a file. 

In [4]:
def preprossed_data(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    X_mag = np.abs(X)
#     print('X_mag',X_mag.shape)
    return X, X_mag

### Save Train Data  

Here we open the files, and read the Train Dataset. We need to run this porion for the first time only. After we 

In [ ]:
# DATA_train_s = None
# DATA_train_n = None
# DATA_train_x = None

with open(TRAIN_S,'wb') as fs, open(TRAIN_N,'wb') as fn, open(TRAIN_X_cmplx,'wb') as fx_cmplx, open(TRAIN_X,'wb') as fx: 
    
    for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_train+CLEAN_format_train)),sorted(glob.glob(PATH_directory+PATH_train+NOISE_format_train)),sorted(glob.glob(PATH_directory+PATH_train+MIX_format_train))):
        _,s = preprossed_data(file_s)
        np.savetxt(fs, s, fmt='%.5f')
        fs.write(b'\n')
        
        _,n = preprossed_data(file_n)
        np.savetxt(fn, n, fmt='%.5f')
        fn.write(b'\n')
        
        x_cmplx,x = preprossed_data(file_x)
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
#         DATA_train_s = np.array(preprossed_data(file_s)) if DATA_train_s is None else np.concatenate( (DATA_train_s,preprossed_data(file_s)),axis=1)
#         DATA_train_n = np.array(preprossed_data(file_n)) if DATA_train_n is None else np.concatenate( (DATA_train_n,preprossed_data(file_n)),axis=1)
#         DATA_train_x = np.array(preprossed_data(file_x)) if DATA_train_x is None else np.concatenate( (DATA_train_x,preprossed_data(file_x)),axis=1)

fs.close()
fn.close()
fx_cmplx.close()
fx.close()

### Save Validate Data

In [ ]:
DATA_val_s = []
DATA_val_s_cmplx = []
DATA_val_n = []
DATA_val_x_cmplx = []
DATA_val_x = []

with open(VAL_S,'wb') as fs, open(VAL_N,'wb') as fn, open(VAL_X_cmplx,'wb') as fx_cmplx, open(VAL_X,'wb') as fx, open(VAL_S_cmplx,'wb') as fs_cmplx: 
    count = 1
    for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)),sorted(glob.glob(PATH_directory+PATH_val+NOISE_format_val)),sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
        s_cmplx,s = preprossed_data(file_s)
        DATA_val_s.append(np.array(s))
        np.savetxt(fs, s, fmt='%.5f')
        fs.write(b'\n')
        
        DATA_val_s_cmplx.append(np.array(s_cmplx))
        np.savetxt(fs_cmplx, s_cmplx, fmt='%.5f')
        fs_cmplx.write(b'\n')
        
        _,n = preprossed_data(file_n)
        DATA_val_n.append(np.array(n))
        np.savetxt(fn, n, fmt='%.5f')
        fn.write(b'\n')
        
        x_cmplx,x = preprossed_data(file_x)
        DATA_val_x.append(np.array(x))
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        DATA_val_x_cmplx.append(np.array(x_cmplx))
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
        print(count, '...')
        count += 1
        
fs.close()
fs_cmplx.close()
fn.close()
fx_cmplx.close()
fx.close()

1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...
10 ...
11 ...
12 ...
13 ...
14 ...
15 ...
16 ...
17 ...
18 ...
19 ...
20 ...
21 ...
22 ...
23 ...
24 ...
25 ...
26 ...
27 ...
28 ...
29 ...
30 ...
31 ...
32 ...
33 ...
34 ...
35 ...
36 ...
37 ...
38 ...
39 ...
40 ...
41 ...
42 ...
43 ...
44 ...
45 ...
46 ...
47 ...
48 ...
49 ...
50 ...
51 ...
52 ...
53 ...
54 ...
55 ...
56 ...
57 ...
58 ...
59 ...
60 ...
61 ...
62 ...
63 ...
64 ...
65 ...
66 ...
67 ...
68 ...
69 ...
70 ...
71 ...
72 ...
73 ...
74 ...
75 ...
76 ...
77 ...
78 ...
79 ...
80 ...
81 ...
82 ...
83 ...
84 ...
85 ...
86 ...
87 ...
88 ...
89 ...
90 ...
91 ...
92 ...
93 ...
94 ...
95 ...
96 ...
97 ...
98 ...
99 ...
100 ...
101 ...
102 ...
103 ...
104 ...
105 ...
106 ...
107 ...
108 ...
109 ...
110 ...
111 ...
112 ...
113 ...
114 ...
115 ...
116 ...
117 ...
118 ...
119 ...
120 ...
121 ...
122 ...
123 ...
124 ...
125 ...
126 ...
127 ...
128 ...
129 ...
130 ...
131 ...
132 ...
133 ...
134 ...
135 ...
136 ...
137 ...
138 ...
139 

### Save Test Data

In [10]:
DATA_test_x_cmplx = []
DATA_test_x = []

with open(TEST_S,'wb') as fs, open(TEST_N,'wb') as fn, open(TEST_X_cmplx,'wb') as fx_cmplx, open(TEST_X,'wb') as fx: 
    count = 1
    for file_x in sorted(glob.glob(PATH_directory+PATH_test+MIX_format_test)):
        x_cmplx,x = preprossed_data(file_x)
        DATA_test_x.append(np.array(x))
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        DATA_test_x_cmplx.append(np.array(x_cmplx))
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
        print(count,'...')
        count += 1
        
fx_cmplx.close()
fx.close()

1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...
10 ...
11 ...
12 ...
13 ...
14 ...
15 ...
16 ...
17 ...
18 ...
19 ...
20 ...
21 ...
22 ...
23 ...
24 ...
25 ...
26 ...
27 ...
28 ...
29 ...
30 ...
31 ...
32 ...
33 ...
34 ...
35 ...
36 ...
37 ...
38 ...
39 ...
40 ...
41 ...
42 ...
43 ...
44 ...
45 ...
46 ...
47 ...
48 ...
49 ...
50 ...
51 ...
52 ...
53 ...
54 ...
55 ...
56 ...
57 ...
58 ...
59 ...
60 ...
61 ...
62 ...
63 ...
64 ...
65 ...
66 ...
67 ...
68 ...
69 ...
70 ...
71 ...
72 ...
73 ...
74 ...
75 ...
76 ...
77 ...
78 ...
79 ...
80 ...
81 ...
82 ...
83 ...
84 ...
85 ...
86 ...
87 ...
88 ...
89 ...
90 ...
91 ...
92 ...
93 ...
94 ...
95 ...
96 ...
97 ...
98 ...
99 ...
100 ...
101 ...
102 ...
103 ...
104 ...
105 ...
106 ...
107 ...
108 ...
109 ...
110 ...
111 ...
112 ...
113 ...
114 ...
115 ...
116 ...
117 ...
118 ...
119 ...
120 ...
121 ...
122 ...
123 ...
124 ...
125 ...
126 ...
127 ...
128 ...
129 ...
130 ...
131 ...
132 ...
133 ...
134 ...
135 ...
136 ...
137 ...
138 ...
139 

### Load Data

In [6]:
def load_file(file_name):
    
    with open(file_name) as f:
        lines=f.readlines()
        print(len(lines))
        sentence_full=[]
        count = 0
        sentence=[]
        for line in lines:

            if count < 513:
                if count ==0:
                    sentence=np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    count+=1
                else:
                    myarray = np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    sentence=np.concatenate((sentence, myarray), axis=0)
                    count+=1
            else:
                sentence_full.append(sentence) 
                count=0
                sentence=[]
                
        return sentence_full

In [7]:
DATA_train_s = load_file(TRAIN_S)
DATA_train_n = load_file(TRAIN_N)
DATA_train_x_cmplx = load_file(TRAIN_X_cmplx)
DATA_train_x = load_file(TRAIN_X)

616800
616800
616800
616800


In [7]:
DATA_train_s[0].shape

(513, 65)

In [6]:
DATA_val_s = load_file(VAL_S)
DATA_val_n = load_file(VAL_N)
DATA_val_x_cmplx = load_file(VAL_X_cmplx)
DATA_val_x = load_file(VAL_X)

616800
616800
616800
616800


In [11]:
DATA_test_x_cmplx = load_file(TEST_X_cmplx)
DATA_test_x = load_file(TEST_X)

205600
205600


In [12]:
DATA_train_M=[ (1.0*(DATA_train_s[i]>DATA_train_n[i])) for i in range(len(DATA_train_s)) ]

DATA_val_M=[ (1.0*(DATA_val_s[i]>DATA_val_n[i])) for i in range(len(DATA_val_s)) ]
    

### Batch Generator

In [13]:
def next_batch_mb(X_,Y_,mini_batch):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
#         print(e)
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
#         print('e',e,'batch_x',batch_x.shape,'batch_y',batch_y.shape)
#         print('batch_y',batch_y.shape)
        
        if e>0 and (e+1)%10==0:
#             print('ZZZ--> e',e,'batch_x',batch_x.shape,'batch_y',batch_y.shape)
            
            batch_x, temp_x = None, batch_x
            batch_y, temp_y = None, batch_y
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))
            
            total_mini_batch = temp_x.shape[0]//mini_batch
#             print('temp_x.shape[0]',temp_x.shape[0], 'total_mini_batch',total_mini_batch)
            
            for mb in range(total_mini_batch):
                start_b = (mb*mini_batch)
                end_b = ((mb+1)*mini_batch)

                yield temp_x[start_b:end_b],temp_y[start_b:end_b]

    

In [14]:
def next_batch(X_,Y_):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
#         print(e)
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
#         print('batch_x',batch_x.shape)
#         print('batch_y',batch_y.shape)
        
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_y, batch_y = batch_y, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))

#             print('temp_x',temp_x.shape)
#             print('temp_y',temp_y.shape)
        
            yield temp_x,temp_y


len(DATA_)

In [48]:
# fXX = np.loadtxt(TEST_X_cmplx, dtype = np.complex128)


with open(TEST_X_cmplx) as f:
        lines=f.readlines()
        for line in lines:
            fXX = np.fromstring(line, dtype=np.complex128, sep=' ')
            print(fXX.shape)

ValueError: don't know how to read character strings with that array type

In [11]:
c=0
for p,q in next_batch(DATA_train_x, DATA_train_M):
    print('p',p.shape)
#     print(p)
    print('q',q.shape)
#     print(q)
    c+=1
    
    if c == 2:
        break
print(c)

batch_x (65, 513)
batch_y (65, 513)
batch_x (130, 513)
batch_y (130, 513)
batch_x (195, 513)
batch_y (195, 513)
batch_x (260, 513)
batch_y (260, 513)
batch_x (325, 513)
batch_y (325, 513)
batch_x (390, 513)
batch_y (390, 513)
batch_x (455, 513)
batch_y (455, 513)
batch_x (520, 513)
batch_y (520, 513)
batch_x (585, 513)
batch_y (585, 513)
batch_x (650, 513)
batch_y (650, 513)
p (130, 5, 513)
q (130, 5, 513)
batch_x (130, 513)
batch_y (130, 513)
batch_x (260, 513)
batch_y (260, 513)
batch_x (390, 513)
batch_y (390, 513)
batch_x (520, 513)
batch_y (520, 513)
batch_x (650, 513)
batch_y (650, 513)
batch_x (780, 513)
batch_y (780, 513)
batch_x (910, 513)
batch_y (910, 513)
batch_x (1040, 513)
batch_y (1040, 513)
batch_x (1170, 513)
batch_y (1170, 513)
batch_x (1300, 513)
batch_y (1300, 513)
p (260, 5, 513)
q (260, 5, 513)
2


In [20]:
len(DATA_train_x)
DATA_train_x[12].shape

(513, 130)

In [15]:
model = Sequential()

model.add(Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(Max_RNN, return_sequences=True)))
# model.add(GRU(output_dim = 513, input_length = 5, input_dim = 513, return_sequences=True))

# model.add(Activation('relu'))
# model.add(TimeDistributed(Dense(513, activation='sigmoid')))
model.add(Dense(513, activation='sigmoid'))

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

for e in range(5):
    for (b_x,b_y), (v_x,v_y) in zip(next_batch(DATA_train_x, DATA_train_M), next_batch(DATA_val_x, DATA_val_M)):
        model.fit(b_x, b_y, validation_data=(v_x,v_y), shuffle=True, batch_size=256)

#     model.fit( , epochs=20, steps_per_epoch=700, validation_data=next_batch_mb(DATA_val_x, DATA_val_x,10), validation_steps=700)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 5, 10)             15570     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 10)             480       
_________________________________________________________________
dense_1 (Dense)              (None, 5, 513)            5643      
Total params: 21,693
Trainable params: 21,693
Non-trainable params: 0
_________________________________________________________________
None
Train on 130 samples, validate on 130 samples
Epoch 1/1
130/130 [==============================] - 3s 24ms/step - loss: 0.2500 - acc: 0.0031 - val_loss: 0.2499 - val_acc: 0.0031
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [======================

212/212 [==============================] - 0s 250us/step - loss: 0.2441 - acc: 0.0028 - val_loss: 0.2502 - val_acc: 0.0019
Train on 210 samples, validate on 210 samples
Epoch 1/1
210/210 [==============================] - 0s 232us/step - loss: 0.2456 - acc: 0.0019 - val_loss: 0.2445 - val_acc: 0.0029
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 302us/step - loss: 0.2382 - acc: 0.0013 - val_loss: 0.2437 - val_acc: 0.0025
Train on 198 samples, validate on 198 samples
Epoch 1/1
198/198 [==============================] - 0s 253us/step - loss: 0.2399 - acc: 0.0061 - val_loss: 0.2417 - val_acc: 0.0010
Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 245us/step - loss: 0.2494 - acc: 0.0019 - val_loss: 0.2450 - val_acc: 9.7087e-04
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 262us/step - loss: 0.2416 - acc: 0.0044 - val_loss: 0.2

Train on 140 samples, validate on 140 samples
Epoch 1/1
140/140 [==============================] - 0s 322us/step - loss: 0.2446 - acc: 0.0014 - val_loss: 0.2373 - val_acc: 0.0014
Train on 228 samples, validate on 228 samples
Epoch 1/1
228/228 [==============================] - 0s 225us/step - loss: 0.2460 - acc: 8.7719e-04 - val_loss: 0.2446 - val_acc: 8.7719e-04
Train on 202 samples, validate on 202 samples
Epoch 1/1
202/202 [==============================] - 0s 263us/step - loss: 0.2407 - acc: 0.0000e+00 - val_loss: 0.2381 - val_acc: 0.0020
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 260us/step - loss: 0.2378 - acc: 0.0033 - val_loss: 0.2371 - val_acc: 0.0022
Train on 242 samples, validate on 242 samples
Epoch 1/1
242/242 [==============================] - 0s 211us/step - loss: 0.2537 - acc: 0.0041 - val_loss: 0.2533 - val_acc: 0.0025
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [============================

Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 377us/step - loss: 0.2236 - acc: 0.0205 - val_loss: 0.2175 - val_acc: 0.0231
Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 333us/step - loss: 0.2213 - acc: 0.0117 - val_loss: 0.2234 - val_acc: 0.0117
Train on 262 samples, validate on 262 samples
Epoch 1/1
262/262 [==============================] - 0s 342us/step - loss: 0.2415 - acc: 0.0374 - val_loss: 0.2387 - val_acc: 0.0267
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 304us/step - loss: 0.2191 - acc: 0.0064 - val_loss: 0.2187 - val_acc: 0.0218
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 292us/step - loss: 0.2378 - acc: 0.0038 - val_loss: 0.2541 - val_acc: 0.0025
Train on 188 samples, validate on 188 samples
Epoch 1/1
188/188 [==============================] - 0s 257

Train on 152 samples, validate on 152 samples
Epoch 1/1
152/152 [==============================] - 0s 369us/step - loss: 0.2242 - acc: 0.0316 - val_loss: 0.2279 - val_acc: 0.0408
Train on 118 samples, validate on 118 samples
Epoch 1/1
118/118 [==============================] - 0s 362us/step - loss: 0.2243 - acc: 0.0085 - val_loss: 0.2267 - val_acc: 0.0136
Train on 232 samples, validate on 232 samples
Epoch 1/1
232/232 [==============================] - 0s 240us/step - loss: 0.2298 - acc: 0.0216 - val_loss: 0.2327 - val_acc: 0.0190
Train on 150 samples, validate on 150 samples
Epoch 1/1
150/150 [==============================] - 0s 291us/step - loss: 0.2089 - acc: 0.0333 - val_loss: 0.2127 - val_acc: 0.0253
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 313us/step - loss: 0.2343 - acc: 0.0125 - val_loss: 0.2307 - val_acc: 0.0222
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 304

...

Train on 214 samples, validate on 214 samples
Epoch 1/1
214/214 [==============================] - 0s 238us/step - loss: 0.2121 - acc: 0.0037 - val_loss: 0.2141 - val_acc: 0.0019
Train on 186 samples, validate on 186 samples
Epoch 1/1
186/186 [==============================] - 0s 267us/step - loss: 0.2148 - acc: 0.0000e+00 - val_loss: 0.1996 - val_acc: 0.0065
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 276us/step - loss: 0.2068 - acc: 0.0033 - val_loss: 0.2025 - val_acc: 0.0033
Train on 222 samples, validate on 222 samples
Epoch 1/1
222/222 [==============================] - 0s 236us/step - loss: 0.2101 - acc: 0.0027 - val_loss: 0.2064 - val_acc: 0.0027
Train on 296 samples, validate on 296 samples
Epoch 1/1
296/296 [==============================] - 0s 323us/step - loss: 0.2012 - acc: 6.7568e-04 - val_loss: 0.1949 - val_acc: 6.7568e-04
Train on 230 samples, validate on 230 samples
Epoch 1/1
230/230 [============================

Train on 356 samples, validate on 356 samples
Epoch 1/1
356/356 [==============================] - 0s 425us/step - loss: 0.1989 - acc: 0.0000e+00 - val_loss: 0.1987 - val_acc: 0.0011
Train on 234 samples, validate on 234 samples
Epoch 1/1
234/234 [==============================] - 0s 256us/step - loss: 0.1893 - acc: 8.5470e-04 - val_loss: 0.1764 - val_acc: 0.0000e+00
Train on 184 samples, validate on 184 samples
Epoch 1/1
184/184 [==============================] - 0s 303us/step - loss: 0.1882 - acc: 0.0022 - val_loss: 0.1927 - val_acc: 0.0022
Train on 238 samples, validate on 238 samples
Epoch 1/1
238/238 [==============================] - 0s 260us/step - loss: 0.2002 - acc: 0.0017 - val_loss: 0.1868 - val_acc: 8.4034e-04
Train on 340 samples, validate on 340 samples
Epoch 1/1
340/340 [==============================] - 0s 324us/step - loss: 0.1954 - acc: 5.8824e-04 - val_loss: 0.1840 - val_acc: 5.8824e-04
Train on 220 samples, validate on 220 samples
Epoch 1/1
220/220 [================

188/188 [==============================] - 0s 414us/step - loss: 0.2017 - acc: 0.0021 - val_loss: 0.1984 - val_acc: 0.0074
Train on 216 samples, validate on 216 samples
Epoch 1/1
216/216 [==============================] - 0s 540us/step - loss: 0.1998 - acc: 9.2593e-04 - val_loss: 0.1857 - val_acc: 0.0000e+00
Train on 176 samples, validate on 176 samples
Epoch 1/1
176/176 [==============================] - 0s 299us/step - loss: 0.1924 - acc: 0.0011 - val_loss: 0.2006 - val_acc: 0.0011
Train on 212 samples, validate on 212 samples
Epoch 1/1
212/212 [==============================] - 0s 240us/step - loss: 0.1900 - acc: 0.0028 - val_loss: 0.1886 - val_acc: 9.4340e-04
Train on 330 samples, validate on 330 samples
Epoch 1/1
330/330 [==============================] - 0s 326us/step - loss: 0.2141 - acc: 0.0012 - val_loss: 0.2079 - val_acc: 0.0018
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 325us/step - loss: 0.2158 - acc: 0.0000e+00 - v

Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 343us/step - loss: 0.1923 - acc: 0.0026 - val_loss: 0.1776 - val_acc: 0.0051
Train on 212 samples, validate on 212 samples
Epoch 1/1
212/212 [==============================] - 0s 350us/step - loss: 0.2004 - acc: 9.4340e-04 - val_loss: 0.1915 - val_acc: 0.0028
Train on 194 samples, validate on 194 samples
Epoch 1/1
194/194 [==============================] - 0s 369us/step - loss: 0.1929 - acc: 0.0000e+00 - val_loss: 0.1862 - val_acc: 0.0021
Train on 164 samples, validate on 164 samples
Epoch 1/1
164/164 [==============================] - 0s 424us/step - loss: 0.2115 - acc: 0.0000e+00 - val_loss: 0.1937 - val_acc: 0.0012
Train on 346 samples, validate on 346 samples
Epoch 1/1
346/346 [==============================] - 0s 395us/step - loss: 0.1981 - acc: 0.0023 - val_loss: 0.1887 - val_acc: 0.0012
Train on 172 samples, validate on 172 samples
Epoch 1/1
172/172 [============================

200/200 [==============================] - 0s 250us/step - loss: 0.1907 - acc: 0.0000e+00 - val_loss: 0.1809 - val_acc: 0.0020
Train on 160 samples, validate on 160 samples
Epoch 1/1
160/160 [==============================] - 0s 317us/step - loss: 0.1979 - acc: 0.0050 - val_loss: 0.1719 - val_acc: 0.0025
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 297us/step - loss: 0.2060 - acc: 0.0038 - val_loss: 0.1848 - val_acc: 0.0038
Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 336us/step - loss: 0.2140 - acc: 0.0026 - val_loss: 0.2036 - val_acc: 0.0026
Train on 166 samples, validate on 166 samples
Epoch 1/1
166/166 [==============================] - 0s 391us/step - loss: 0.1906 - acc: 0.0012 - val_loss: 0.1841 - val_acc: 0.0024
Train on 224 samples, validate on 224 samples
Epoch 1/1
224/224 [==============================] - 0s 341us/step - loss: 0.1830 - acc: 0.0054 - val_loss: 0.1

In [17]:
# Final evaluation of the model
scores = []
for v_x,v_y in next_batch(DATA_val_x, DATA_val_x):
    scores.append( model.evaluate(v_x, v_y,verbose=0)[1] )

print("Accuracy: %.2f%%" % np.mean(scores))

Accuracy: 0.01%


In [23]:
def next_batch_SXX_cmplxS_cmplx(S_,X_,X_cmplx_,S_cmplx_):
    
    batch_s = None
    batch_x = None
    batch_x_cmplx = None
    batch_s_cmplx = None
    
    for e,(s,x,x_cmplx,s_cmplx) in enumerate( zip(S_,X_,X_cmplx_,S_cmplx_)): 
        batch_s = np.array(s.T) if batch_s is None else np.concatenate( (batch_s,s.T), axis=0)
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_x_cmplx = np.array(x_cmplx.T) if batch_x_cmplx is None else np.concatenate( (batch_x_cmplx,x_cmplx.T), axis=0)
        batch_s_cmplx = np.array(s_cmplx.T) if batch_s_cmplx is None else np.concatenate( (batch_s_cmplx,s_cmplx.T), axis=0)
        
        
        if e>0 and (e+1)%10==0:
            temp_s, batch_s = batch_s, None
            temp_x, batch_x = batch_x, None
            temp_x_cmplx, batch_x_cmplx = batch_x_cmplx, None
            temp_s_cmplx, batch_s_cmplx = batch_s_cmplx, None
            
            temp_s = temp_s.reshape((-1,Max_RNN,513))
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_x_cmplx = temp_x_cmplx.reshape((-1,Max_RNN,513))

            yield temp_s,temp_x,temp_x_cmplx,temp_s_cmplx

In [26]:
sum_s = 0.0
sum_s_diff = 0.0


for v_s,v_x,v_x_cmplx,v_s_cmplx in next_batch_SXX_cmplxS_cmplx(DATA_val_s,DATA_val_x,DATA_val_x_cmplx,DATA_val_s_cmplx):
    
#     print(v_s.shape)
#     print(v_x.shape)
#     print(v_x_cmplx.shape)
#     print(v_s_cmplx.shape)
    
    mask = model.predict(v_x)
    S_hat = (mask) * v_x_cmplx
    S_hat = S_hat.reshape(-1,513).T
    S = v_s_cmplx.T
    
#     print(S.shape)
#     print(S_hat.shape)

    S_org = librosa.istft(S, hop_length=512)
    S_pred = librosa.istft(S_hat, hop_length=512)

    sum_s += np.sum(S_org*S_org)
    sum_s_diff += np.sum((S_org-S_pred)*(S_org-S_pred))
    
acc = sum_s/ sum_s_diff
print(acc)

10*np.log10(acc)

(130, 5, 513)
(130, 5, 513)
(130, 5, 513)
(650, 513)
(513, 650)
(513, 650)
(260, 5, 513)
(260, 5, 513)
(260, 5, 513)
(1300, 513)
(513, 1300)
(513, 1300)
(150, 5, 513)
(150, 5, 513)
(150, 5, 513)
(750, 513)
(513, 750)
(513, 750)
(138, 5, 513)
(138, 5, 513)
(138, 5, 513)
(690, 513)
(513, 690)
(513, 690)
(184, 5, 513)
(184, 5, 513)
(184, 5, 513)
(920, 513)
(513, 920)
(513, 920)
(206, 5, 513)
(206, 5, 513)
(206, 5, 513)
(1030, 513)
(513, 1030)
(513, 1030)
(174, 5, 513)
(174, 5, 513)
(174, 5, 513)
(870, 513)
(513, 870)
(513, 870)
(278, 5, 513)
(278, 5, 513)
(278, 5, 513)
(1390, 513)
(513, 1390)
(513, 1390)
(156, 5, 513)
(156, 5, 513)
(156, 5, 513)
(780, 513)
(513, 780)
(513, 780)
(160, 5, 513)
(160, 5, 513)
(160, 5, 513)
(800, 513)
(513, 800)
(513, 800)
(282, 5, 513)
(282, 5, 513)
(282, 5, 513)
(1410, 513)
(513, 1410)
(513, 1410)
(138, 5, 513)
(138, 5, 513)
(138, 5, 513)
(690, 513)
(513, 690)
(513, 690)
(156, 5, 513)
(156, 5, 513)
(156, 5, 513)
(780, 513)
(513, 780)
(513, 780)
(154, 5, 513)

(513, 1000)
(513, 1000)
(160, 5, 513)
(160, 5, 513)
(160, 5, 513)
(800, 513)
(513, 800)
(513, 800)
(158, 5, 513)
(158, 5, 513)
(158, 5, 513)
(790, 513)
(513, 790)
(513, 790)
(154, 5, 513)
(154, 5, 513)
(154, 5, 513)
(770, 513)
(513, 770)
(513, 770)
(166, 5, 513)
(166, 5, 513)
(166, 5, 513)
(830, 513)
(513, 830)
(513, 830)
(224, 5, 513)
(224, 5, 513)
(224, 5, 513)
(1120, 513)
(513, 1120)
(513, 1120)
(294, 5, 513)
(294, 5, 513)
(294, 5, 513)
(1470, 513)
(513, 1470)
(513, 1470)
(150, 5, 513)
(150, 5, 513)
(150, 5, 513)
(750, 513)
(513, 750)
(513, 750)
(196, 5, 513)
(196, 5, 513)
(196, 5, 513)
(980, 513)
(513, 980)
(513, 980)
(148, 5, 513)
(148, 5, 513)
(148, 5, 513)
(740, 513)
(513, 740)
(513, 740)
(234, 5, 513)
(234, 5, 513)
(234, 5, 513)
(1170, 513)
(513, 1170)
(513, 1170)
(180, 5, 513)
(180, 5, 513)
(180, 5, 513)
(900, 513)
(513, 900)
(513, 900)
3.140206685271807


4.96958233843578

In [37]:
def write_audio():
    mags = None
    
    for e, file_x in enumerate(sorted(glob.glob(PATH_directory+PATH_test+MIX_format_val))):
        print(e)
        sn, sr = librosa.load(file_x, sr=None)
        Sn = librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
        
        mags = np.array(mag_Sn.T) if mags is None else np.concatenate( (mags,mag_Sn.T), axis=0)
        
        if e>0 and (e+1)%10==0:
            temp, mags = mags, None
            
            temp = temp.reshape((-1,Max_RNN,513))
            mask = model.predict(temp)
            
            S_hat = (mask) * Sn
            S_hat = S_hat.reshape(-1,513).T
            
            lenght_w = S_hat.shape[1]/10
            for clip in range(10):
                start_w = clip*lenght_w
                end_w = (clip+1)*lenght_w
                
                wav = S_hat[:,start_w:end_w].T
                S_time=librosa.istft(wav, hop_length=512)
                fname = PATH_directory+PATH_denoise+ e + "_redoise.wav"
                
                print(audio_fname)
                librosa.output.write_wav(fname, S_time, sr)
                

In [38]:
write_audio()

Create IRM for training.

print('DATA_train_s',DATA_train_s.shape)
print('DATA_train_n',DATA_train_n.shape)
print('DATA_train_x',DATA_train_x.shape)

DATA_train_s = DATA_train_s.reshape( (513,5,-1))
DATA_train_n = DATA_train_n.reshape( (513,5,-1))
DATA_train_x = DATA_train_x.reshape( (513,5,-1))
DATA_train_M = 1*(DATA_train_s>DATA_train_n)

print('DATA_train_s',DATA_train_s.shape)
print('DATA_train_n',DATA_train_n.shape)
print('DATA_train_x',DATA_train_x.shape)
print('DATA_train_M',DATA_train_M.shape)

### Validate Data

DATA_val_s = None
DATA_val_n = None
DATA_val_x = None

for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)),sorted(glob.glob(PATH_directory+PATH_val+NOISE_format_val)),sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
    
    DATA_val_s = np.array(preprossed_data(file_s)) if DATA_val_s is None else np.concatenate( (DATA_val_s,preprossed_data(file_s)),axis=1)
    DATA_val_n = np.array(preprossed_data(file_n)) if DATA_val_n is None else np.concatenate( (DATA_val_n,preprossed_data(file_n)),axis=1)
    DATA_val_x = np.array(preprossed_data(file_x)) if DATA_val_x is None else np.concatenate( (DATA_val_x,preprossed_data(file_x)),axis=1)

print('DATA_val_s',DATA_val_s.shape)
print('DATA_val_n',DATA_val_n.shape)
print('DATA_val_x',DATA_val_x.shape)

DATA_val_s = DATA_val_s.reshape( (513,5,-1))
DATA_val_n = DATA_val_n.reshape( (513,5,-1))
DATA_val_x = DATA_val_x.reshape( (513,5,-1))
DATA_val_M = 1*(DATA_val_s>DATA_val_n)

print('DATA_val_s',DATA_val_s.shape)
print('DATA_val_n',DATA_val_n.shape)
print('DATA_val_x',DATA_val_x.shape)
print('DATA_val_M',DATA_val_M.shape)

### Test Data

DATA_test_x = None

for file_x in sorted(glob.glob(PATH_directory+PATH_test+MIX_format_test)):
    DATA_test_x = np.array(preprossed_data(file_x)) if DATA_test_x is None else np.concatenate( (DATA_test_x,preprossed_data(file_x)),axis=1)


In [ ]:
print('DATA_test_x',DATA_test_x.shape)
DATA_test_x = DATA_test_x.reshape( (513,5,-1))

print('DATA_test_x',DATA_test_x.shape)

In [7]:
def preprossed_dataX(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    return X

### Validate Data Complex

In [8]:
DATA_val_sX = None
DATA_val_xX = None

for file_s, file_x in zip( sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)), sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
    
    DATA_val_sX = np.array(preprossed_dataX(file_s)) if DATA_val_sX is None else np.concatenate( (DATA_val_sX,preprossed_dataX(file_s)),axis=1)
    DATA_val_xX = np.array(preprossed_dataX(file_x)) if DATA_val_xX is None else np.concatenate( (DATA_val_xX,preprossed_dataX(file_x)),axis=1)
    
    
    
   

In [9]:
print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_sX = DATA_val_sX.reshape( (513,5,-1))
DATA_val_xX = DATA_val_xX.reshape( (513,5,-1))

print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_sX (513, 118550)
DATA_val_xX (513, 118550)
DATA_val_sX (513, 5, 23710)
DATA_val_xX (513, 5, 23710)


In [10]:
np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

In [50]:
DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

DATA_test_x (513, 44150)
DATA_test_x (513, 5, 8830)


### Save Dataset in files

In [3]:
np.save('DATA_train_s.npy', DATA_train_s)
np.save('DATA_train_n.npy', DATA_train_n)
np.save('DATA_train_x.npy', DATA_train_x)
np.save('DATA_train_M.npy', DATA_train_M)


np.save('DATA_val_s.npy', DATA_val_s)
np.save('DATA_val_n.npy', DATA_val_n)
np.save('DATA_val_x.npy', DATA_val_x)
np.save('DATA_val_M.npy', DATA_val_M)

np.save('DATA_test_x.npy', DATA_test_x)

np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

NameError: name 'DATA_train_s' is not defined

### Load Dataset in files

In [3]:
DATA_train_s_f = np.load('DATA_train_s.npy')
DATA_train_n_f = np.load('DATA_train_n.npy')
DATA_train_x_f = np.load('DATA_train_x.npy')
DATA_train_M_f = np.load('DATA_train_M.npy')

DATA_val_s_f = np.load('DATA_val_s.npy')
DATA_val_n_f = np.load('DATA_val_n.npy')
DATA_val_x_f = np.load('DATA_val_x.npy')
DATA_val_M_f = np.load('DATA_val_M.npy')

DATA_test_x_f = np.load('DATA_test_x.npy')

DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

In [4]:
print('DATA_train_s_f', DATA_train_s_f.shape)
print('DATA_train_M_f', DATA_train_M_f.shape)
print('DATA_val_x_f', DATA_val_x_f.shape)
print('DATA_test_x_f', DATA_test_x_f.shape)
print('DATA_val_xX_f', DATA_val_xX_f.shape)

DATA_train_s_f (513, 5, 23710)
DATA_train_M_f (513, 5, 23710)
DATA_val_x_f (513, 5, 23710)
DATA_test_x_f (513, 5, 8830)
DATA_val_xX_f (513, 5, 23710)


In [5]:
DATA_train_s_f = DATA_train_s_f.reshape(513,-1)
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f = DATA_train_s_f.T
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f = DATA_train_s_f.reshape(-1,5,513)
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f (513, 118550)
DATA_train_s_f (118550, 513)
DATA_train_s_f (23710, 5, 513)


In [6]:
DATA_train_n_f = (DATA_train_n_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_train_x_f = (DATA_train_x_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_train_M_f = (DATA_train_M_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_val_s_f = (DATA_val_s_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_n_f = (DATA_val_n_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_x_f = (DATA_val_x_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_M_f = (DATA_val_M_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_test_x_f = (DATA_test_x_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_val_sX_f = (DATA_val_sX_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_xX_f = (DATA_val_xX_f.reshape(513,-1).T).reshape(-1,5,513)

In [7]:
print('DATA_train_s_f', DATA_train_s_f.shape)
print('DATA_train_M_f', DATA_train_M_f.shape)
print('DATA_val_x_f', DATA_val_x_f.shape)
print('DATA_test_x_f', DATA_test_x_f.shape)
print('DATA_val_xX_f', DATA_val_xX_f.shape)

DATA_train_s_f (23710, 5, 513)
DATA_train_M_f (23710, 5, 513)
DATA_val_x_f (23710, 5, 513)
DATA_test_x_f (8830, 5, 513)
DATA_val_xX_f (23710, 5, 513)


In [22]:
tr_categorical_labels = to_categorical(DATA_train_M_f, num_classes=2)
print(tr_categorical_labels.shape)

v_categorical_labels = to_categorical(DATA_val_M_f, num_classes=2)
print(v_categorical_labels.shape)

(23710, 5, 513, 2)
(23710, 5, 513, 2)


### RNN Model

In [8]:
Max_RNN = 5

In [10]:
# create the model

model = Sequential()

model.add( Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513) ))
model.add(Dropout(0.1))
model.add(Dense(513, activation='sigmoid'))
          
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# model.fit(DATA_train_x_f.T, tr_categorical_labels, validation_data=(DATA_val_x_f.T,v_categorical_labels), shuffle=True, nb_epoch=100, batch_size=10)
# # Final evaluation of the model
# scores = model.evaluate(DATA_val_x_f.T, v_categorical_labels, verbose=0)



model.fit(DATA_train_x_f, DATA_train_M_f, validation_data=(DATA_val_x_f,DATA_val_M_f), shuffle=True, nb_epoch=10, batch_size=10)

# model.fit(DATA_train_x_f, tr_categorical_labels, validation_data=(DATA_val_x_f,v_categorical_labels), shuffle=True, nb_epoch=10, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f, DATA_val_M_f, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 5, 10)             15570     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5, 513)            5643      
Total params: 21,213
Trainable params: 21,213
Non-trainable params: 0
_________________________________________________________________
None


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 23710 samples, validate on 23710 samples
Epoch 1/10
23710/23710 [==============================] - 55s 2ms/step - loss: 0.5898 - acc: 0.6839 - val_loss: 0.5429 - val_acc: 0.7291
Epoch 2/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5468 - acc: 0.7229 - val_loss: 0.5234 - val_acc: 0.7439
Epoch 3/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5346 - acc: 0.7325 - val_loss: 0.5102 - val_acc: 0.7529
Epoch 4/10
23710/23710 [==============================] - 53s 2ms/step - loss: 0.5267 - acc: 0.7385 - val_loss: 0.5061 - val_acc: 0.7552
Epoch 5/10
23710/23710 [==============================] - 53s 2ms/step - loss: 0.5212 - acc: 0.7429 - val_loss: 0.5035 - val_acc: 0.7576
Epoch 6/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5175 - acc: 0.7459 - val_loss: 0.4956 - val_acc: 0.7625
Epoch 7/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5145 - acc: 0.7477 - val_loss: 0.4921 - val_acc

In [ ]:
model = Sequential()

model.add(GRU(output_dim = 513, input_shape=(Max_RNN,513), return_sequences=True))
# model.add(GRU(output_dim = 513, input_length = 5, input_dim = 513, return_sequences=True))

model.add(Activation('relu'))
model.add(TimeDistributed(Dense(513, activation='sigmoid')))
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

model.fit(DATA_train_x_f, DATA_train_M_f, validation_data=(DATA_val_x_f,DATA_val_M_f), shuffle=True, epochs=10, batch_size=10)

# model.fit(DATA_train_x_f, tr_categorical_labels, validation_data=(DATA_val_x_f,v_categorical_labels), shuffle=True, nb_epoch=10, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f, DATA_val_M_f, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(input_shape=(5, 513), return_sequences=True, units=513)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 5, 513)            1580553   
_________________________________________________________________
activation_1 (Activation)    (None, 5, 513)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 513)            263682    
Total params: 1,844,235
Trainable params: 1,844,235
Non-trainable params: 0
_________________________________________________________________
None
Train on 23710 samples, validate on 23710 samples
Epoch 1/10


In [8]:
def preprossed_dataX(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    return X

In [33]:
DATA_val_sX = None
DATA_val_xX = None

for file_s, file_x in zip( sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)), sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
    
    DATA_val_sX = np.array(preprossed_dataX(file_s)) if DATA_val_sX is None else np.concatenate( (DATA_val_sX,preprossed_dataX(file_s)),axis=1)
    DATA_val_xX = np.array(preprossed_dataX(file_x)) if DATA_val_xX is None else np.concatenate( (DATA_val_xX,preprossed_dataX(file_x)),axis=1)
    
    
    
    

In [10]:
print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_sX = DATA_val_sX.reshape( (513,5,-1))
DATA_val_xX = DATA_val_xX.reshape( (513,5,-1))

print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_xX (513, 118550)
DATA_val_xX (513, 5, 23710)


In [ ]:
np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

In [13]:
DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

In [25]:
labels = model.predict(DATA_test_x)
labels.shape

ValueError: Error when checking model : the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 400 arrays: [array([[0.31088, 0.00439, 0.02531, ..., 0.03555, 0.05921, 0.02048],
       [0.20403, 0.21771, 0.24856, ..., 0.07964, 0.19597, 0.11246],
       [0.00054, 0.27443, 0.5295 , ..., 0.14783, 0.20548, 0.119...

In [12]:
S_hat = (labels) * DATA_val_xX_f

In [13]:
# S_org = (DATA_val_s_f).flatten()
# S_pred = S_hat.flatten()
st = DATA_val_sX_f.reshape(513,-1)
st_hat = S_hat.reshape(513,-1)

print(st.shape)
print(st_hat.shape)

S_org = librosa.istft(st, hop_length=512)
S_pred = librosa.istft(st_hat, hop_length=512)

(513, 118550)
(513, 118550)


In [20]:
print(S_pred.shape, S_org.shape)

(60697088,) (60697088,)


In [14]:
acc = np.sum(S_org*S_org)/ np.sum((S_org-S_pred)*(S_org-S_pred))
print(acc)

10*np.log10(acc)

4.1112037


6.139689683914185

In [1]:
# Noisy -> Input Data
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag = np.abs(X)

# Clean -> Label
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
S_mag = np.abs(S)

IndentationError: expected an indented block (<ipython-input-1-09b5a3680f5d>, line 9)

### Function for generating next batch

In [56]:
def next_batch(X,Y, batch_size):
    num_samples, _ = X.shape
    
    selected_indics = np.random.randint(num_samples-batch_size)
#     print(selected_indics)
    return X[selected_indics:selected_indics+batch_size], Y[selected_indics:selected_indics+batch_size]

### Global Parameters

In [57]:
NUM_ITERATION = 1000
BATCH_SIZE = X_mag.shape[0]

### Xavier Initialization of Weights
These are the weight initialization function used in defining model.

In [58]:
def weight_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(2.0/sum(shape)) )
    return tf.Variable(initial)

def bias_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(1.0/sum(shape)) )
    return tf.Variable(initial)

### Create the fully connected model 

In [59]:
x = tf.placeholder(tf.float32, [None, 513]) 

W_1 = weight_variable([513, 1024])
b_1 = bias_variable([1024])

W_2 = weight_variable([1024, 1024])
b_2 = bias_variable([1024])

W_3 = weight_variable([1024, 1024])
b_3 = bias_variable([1024])

W_4 = weight_variable([1024, 1024])
b_4 = bias_variable([1024])

W_5 = weight_variable([1024, 513])
b_5 = bias_variable([513])

y_ = tf.placeholder(tf.float32, [None, 513]) # original


# Layer connections and Activation functions
y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)
y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)
y_3 = tf.nn.relu(tf.matmul(y_2, W_3) + b_3)
y_4 = tf.nn.relu(tf.matmul(y_3, W_4) + b_4)
y =  tf.nn.relu(tf.matmul(y_2, W_5) + b_5) # predicted


# Define loss and optimizer
mse = tf.reduce_sum( tf.losses.mean_squared_error(labels=y_, predictions=y) )
train_step = tf.train.AdamOptimizer().minimize(mse)

### Run the model
We try different batch sizes, but for whole input dimension batch size gives better result and it removes the need of batch normalization. Since the input dimension is not very large, we can do this. For larger input dimensions, we have to adopt mini batch techniques. 

In [60]:
# Configuration to control GPU use
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()


# Train Model
for _ in range(NUM_ITERATION):
    for _ in range(((X_mag.T).shape[0]//BATCH_SIZE)):
        batch_xs, batch_ys = next_batch(X_mag.T,S_mag.T, BATCH_SIZE)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

### Read the Test files

In [61]:
# Load Test data-1
sn, sr=librosa.load('test_x_01.wav', sr=None)
X_test_01=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag_test_01 = np.abs(X_test_01)

# Load Test data-2
sn, sr=librosa.load('test_x_02.wav', sr=None)
X_test_02=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag_test_02 = np.abs(X_test_02)

print(X_mag_test_01.shape)

(513, 142)


### Save the Test Output Files

In [62]:
# Test model-1
S_hat_mag_test_01=sess.run(y, feed_dict={x: X_mag_test_01.T})
S_hat_test_01=(X_test_01/X_mag_test_01)*S_hat_mag_test_01.T
S_hat_01=librosa.istft(S_hat_test_01, hop_length=512)
librosa.output.write_wav('test_s_01_recons.wav', S_hat_01, sr)

# Test model-2
S_hat_mag_test_02=sess.run(y, feed_dict={x: X_mag_test_02.T})
S_hat_test_02=(X_test_02/X_mag_test_02)*S_hat_mag_test_02.T
S_hat_02=librosa.istft(S_hat_test_02, hop_length=512)
librosa.output.write_wav('test_s_02_recons.wav', S_hat_02, sr)